In [139]:
from lusidtools.jupyter_tools import toggle_code

"""Intest Rate Swap with fixed and floating leg

Demonstrates creation of an interest rate swap in LUSID with a 
fixed and floating rate leg.

Attributes
----------
interest rate swap
instruments
"""

toggle_code("Hide docstring")

## Modelling a swap in LUSID using LUSID instrument model

### This notebook demonstrates upserting a swap in LUSID using both the full LUSID instrument model and a simple instrument. We apply this to a case where the source system assigns each swap leg it's own ID

In [140]:
# Use first block to import generic non-LUSID packages
import os
import pandas as pd
import datetime
import json
import pytz
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from datetime import datetime

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "${:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lu.api.InstrumentsApi)
property_definitions_api = api_factory.build(lu.api.PropertyDefinitionsApi)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)


,api_version,build_version,excel_version,links
0,v0,0.6.9287.0,0.5.2863,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [141]:
#Dates, scopes and market data
start = datetime(2022, 1, 4, tzinfo=pytz.utc)
maturity = datetime(2023, 1, 4, tzinfo=pytz.utc)
scope = "ExampleIRSUpsert"
portfolio_code = "SwapPortfolio"

## 1.Create property defintions for leg ID's

In [142]:
try:
    pay_leg_property_request = lm.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope=scope,
        code="payLegId",
        display_name="pay LegId",
        data_type_id=lm.ResourceId(
            scope="system",
            code="string",
        ),
        life_time="Perpetual",
    )

    property_definitions_api.create_property_definition(
        create_property_definition_request=pay_leg_property_request
    )
    receive_leg_property_request = lm.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope=scope,
        code="receiveLegID",
        display_name="receive LegID",
        data_type_id=lm.ResourceId(
            scope="system",
            code="string",
        ),
        life_time="Perpetual",
    )

    property_definitions_api.create_property_definition(
        create_property_definition_request=receive_leg_property_request
    )
except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/ExampleIRSUpsert/payLegId' because it already exists.


## 2. Model using full LUSID instrument model

In [143]:
## Define flow convention for Fixed Leg

flow_conventionFixed = lm.FlowConventions(
    currency="USD",
    payment_frequency="1Y",
    day_count_convention="Act360",
    roll_convention="ModifiedFollowing",
    settle_days=0,
    reset_days=0,
    payment_calendars=[],
    reset_calendars=[]
)

## Define flow convention for floating leg

flow_conventionFloat = lm.FlowConventions(
    currency="USD",
    payment_frequency="1Y",
    day_count_convention="Act360",
    roll_convention="ModifiedFollowing",
    settle_days=0,
    reset_days=0,
    payment_calendars=[],
    reset_calendars=[]
)

float_leg_idx_conv = lm.IndexConvention(
    currency="USD",
    payment_tenor="1D",
    fixing_reference="USD.1D.SOFRFIXING", # this links to instrument_id in quoteseriesID
    publication_day_lag=0,
    day_count_convention="Act360",
    index_name="SOFR"
)

# Create the leg definitions
float_leg_definition = lm.LegDefinition(
    index_convention=float_leg_idx_conv,
    pay_receive="Receive",
    conventions=flow_conventionFloat,
    stub_type="Both",
    notional_exchange_type="None",
    rate_or_spread=0.0025,
    reset_convention="InArrears",
    compounding = ""
)

fixed_leg_definition = lm.LegDefinition(
    rate_or_spread=0.025,
    pay_receive="Pay",
    conventions=flow_conventionFixed,
    stub_type="Both",
    notional_exchange_type="None",
)

# Create the legs
fixed_leg = lm.FixedLeg(
    start_date=start.isoformat(),
    maturity_date=maturity.isoformat(),
    notional=10000000,
    leg_definition=fixed_leg_definition,
    instrument_type="FixedLeg",
    overrides={}
)

floating_leg = lm.FloatingLeg(
    start_date=start.isoformat(),
    maturity_date=maturity.isoformat(),
    notional=10000000,
    leg_definition=float_leg_definition,
    instrument_type="FloatingLeg",
    overrides={}
)

irs_legs = [
    fixed_leg,
    floating_leg
]

payLegIdproperty = lm.ModelProperty(
    key=f"Instrument/{scope}/payLegId",
    value=lm.PropertyValue(label_value= "12345"
    ),
)

receiveLegIDproperty = lm.ModelProperty(
    key=f"Instrument/{scope}/receiveLegID",
    value=lm.PropertyValue(label_value="12346"
    ),
)

# create the swap
instrument_definition_IRS = lm.InterestRateSwap(
    start_date=start.isoformat(),
    maturity_date=maturity.isoformat(),
    legs=irs_legs,
    instrument_type="InterestRateSwap")

IRS_definition = lm.InstrumentDefinition(
    name="InterestRateSwap",
    identifiers={"ClientInternal": lm.InstrumentIdValue("Swap2")},
    definition=instrument_definition_IRS,
    properties=[payLegIdproperty,receiveLegIDproperty],
)

## 3. Modelling as simple instrument

In [144]:
## Simple instruments can be used without a full economic definition

simpleIRS = lm.SimpleInstrument(
        instrument_type="SimpleInstrument",
        dom_ccy="USD",
        asset_class="InterestRates",
        simple_instrument_type="InterestRates",
    )

payLegIdpropertySimple = lm.ModelProperty(
    key=f"Instrument/{scope}/payLegId",
    value=lm.PropertyValue(label_value="99998"
    ),
)

receiveLegIDpropertySimple = lm.ModelProperty(
    key=f"Instrument/{scope}/receiveLegID",
    value=lm.PropertyValue(label_value="99997"
    ),
)


simple_swap_definition = lm.InstrumentDefinition(
    name="MySimpleIRS",
    identifiers={"ClientInternal": lm.InstrumentIdValue("Swap1")},
    definition=simpleIRS,
    properties=[payLegIdpropertySimple,receiveLegIDpropertySimple],
)

In [145]:
# upsert both instruments
upsert_request = {"Swap1": simple_swap_definition, "Swap2" : IRS_definition}
upsert_response = instruments_api.upsert_instruments(request_body=upsert_request, scope=scope)

## 4.Query swap using individual leg ID's

In [146]:
### Return simple instrument using pay leg ID

legfilter = f"properties[Instrument/{scope}/payLegId] eq '99998'"
listofinstruments = instruments_api.list_instruments(scope=scope, filter=legfilter, instrument_property_keys= [f"Instrument/{scope}/payLegId",f"Instrument/{scope}/receiveLegID"])

display(lusid_response_to_data_frame(listofinstruments))

,href,scope,lusid_instrument_id,version.effective_from,version.as_at_date,name,identifiers.ClientInternal,identifiers.LusidInstrumentId,properties.0.key,properties.0.value.label_value,properties.0.effective_from,properties.0.effective_until,properties.1.key,properties.1.value.label_value,properties.1.effective_from,properties.1.effective_until,instrument_definition.maturity_date,instrument_definition.dom_ccy,instrument_definition.asset_class,instrument_definition.fgn_ccys,instrument_definition.simple_instrument_type,instrument_definition.instrument_type,state,asset_class,dom_ccy
0,https://fbn-ci.lusid.com/api/api/instruments/L...,ExampleIRSUpsert,LUID_0001M5GZ,0001-01-01 00:00:00+00:00,2022-05-23 18:27:42.430777+00:00,MySimpleIRS,Swap1,LUID_0001M5GZ,Instrument/ExampleIRSUpsert/receiveLegID,99997,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,Instrument/ExampleIRSUpsert/payLegId,99998,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,9999-12-31 23:59:59.999999+00:00,USD,InterestRates,[],InterestRates,SimpleInstrument,Active,InterestRates,USD


In [147]:
## Return full swap instrument using payLegId

legfilter = f"properties[Instrument/{scope}/payLegId] eq '12345'"
listofinstruments = instruments_api.list_instruments(scope=scope, filter=legfilter, instrument_property_keys= [f"Instrument/{scope}/payLegId",f"Instrument/{scope}/receiveLegID"])
display(lusid_response_to_data_frame(listofinstruments))

,href,scope,lusid_instrument_id,version.effective_from,version.as_at_date,name,identifiers.LusidInstrumentId,identifiers.ClientInternal,properties.0.key,properties.0.value.label_value,properties.0.effective_from,properties.0.effective_until,properties.1.key,properties.1.value.label_value,properties.1.effective_from,properties.1.effective_until,instrument_definition.start_date,instrument_definition.maturity_date,instrument_definition.is_non_deliverable,instrument_definition.legs.0.start_date,instrument_definition.legs.0.maturity_date,instrument_definition.legs.0.leg_definition.conventions.currency,instrument_definition.legs.0.leg_definition.conventions.payment_frequency,instrument_definition.legs.0.leg_definition.conventions.day_count_convention,instrument_definition.legs.0.leg_definition.conventions.roll_convention,instrument_definition.legs.0.leg_definition.conventions.payment_calendars,instrument_definition.legs.0.leg_definition.conventions.reset_calendars,instrument_definition.legs.0.leg_definition.conventions.settle_days,instrument_definition.legs.0.leg_definition.conventions.reset_days,instrument_definition.legs.0.leg_definition.conventions.leap_days_included,instrument_definition.legs.0.leg_definition.notional_exchange_type,instrument_definition.legs.0.leg_definition.pay_receive,instrument_definition.legs.0.leg_definition.rate_or_spread,instrument_definition.legs.0.leg_definition.reset_convention,instrument_definition.legs.0.leg_definition.stub_type,instrument_definition.legs.0.notional,instrument_definition.legs.0.instrument_type,instrument_definition.legs.1.start_date,instrument_definition.legs.1.maturity_date,instrument_definition.legs.1.leg_definition.conventions.currency,instrument_definition.legs.1.leg_definition.conventions.payment_frequency,instrument_definition.legs.1.leg_definition.conventions.day_count_convention,instrument_definition.legs.1.leg_definition.conventions.roll_convention,instrument_definition.legs.1.leg_definition.conventions.payment_calendars,instrument_definition.legs.1.leg_definition.conventions.reset_calendars,instrument_definition.legs.1.leg_definition.conventions.settle_days,instrument_definition.legs.1.leg_definition.conventions.reset_days,instrument_definition.legs.1.leg_definition.conventions.leap_days_included,instrument_definition.legs.1.leg_definition.index_convention.fixing_reference,instrument_definition.legs.1.leg_definition.index_convention.publication_day_lag,instrument_definition.legs.1.leg_definition.index_convention.payment_tenor,instrument_definition.legs.1.leg_definition.index_convention.day_count_convention,instrument_definition.legs.1.leg_definition.index_convention.currency,instrument_definition.legs.1.leg_definition.index_convention.index_name,instrument_definition.legs.1.leg_definition.notional_exchange_type,instrument_definition.legs.1.leg_definition.pay_receive,instrument_definition.legs.1.leg_definition.rate_or_spread,instrument_definition.legs.1.leg_definition.reset_convention,instrument_definition.legs.1.leg_definition.stub_type,instrument_definition.legs.1.notional,instrument_definition.legs.1.instrument_type,instrument_definition.instrument_type,state,asset_class,dom_ccy
0,https://fbn-ci.lusid.com/api/api/instruments/L...,ExampleIRSUpsert,LUID_0001M5H0,0001-01-01 00:00:00+00:00,2022-05-23 18:27:42.430777+00:00,InterestRateSwap,LUID_0001M5H0,Swap2,Instrument/ExampleIRSUpsert/payLegId,12345,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,Instrument/ExampleIRSUpsert/receiveLegID,12346,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,2022-01-04 00:00:00+00:00,2023-01-04 00:00:00+00:00,False,2022-01-04 00:00:00+00:00,2023-01-04 00:00:00+00:00,USD,1Y,Actual360,ModifiedFollowing,[],[],0,0,True,None,Pay,$0.03,InAdvance,ShortBack,"$10,000,000.00",FixedLeg,2022-01-04 00:00:00+00:00,2023-01-04 00:00:00+00:00,USD,1Y,Actual360,ModifiedFollowing,[],[],0,0,True,USD.1D.SOFRFIXING,0,1D,Actual360,USD,SOFR,None,Receive,$0.00,InArrears,ShortBack,"$10,000,000.00",